In [0]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

In [2]:
# check if gpu is available or not

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
	print("[INFO] Training on CPU.......")

else:
	print("[INFO] Training on GPU.......")


[INFO] Training on GPU.......


In [0]:
def display(train_data):

  images , labels = next(iter(train_loader))

  images.numpy()

  fig = plt.figure(figsize = (25,4))
  for idx in np.arange(20):
    ax = fig.add_subplot(2 , 10,  idx + 1, xticks = [], yticks = [])
    plt.imshow(np.transpose(images[idx], (1,2,0)))
    print(ax.set_title(classes[labels[idx]]))
      





In [0]:
data_dir = "drive/My Drive/dataset/train/"

# labels
classes = ["cloudy", "foggy", "rainy", "shine", "sunrise"]


In [0]:

# transforms 
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
									                    transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])	




valid_size = 0.2




# laoding dataset from directory
train_data = datasets.ImageFolder(data_dir, transform = train_transform )



num_train = len(train_data)	
idx = list(range(num_train))	
np.random.shuffle(idx)

split = int(np.floor(valid_size * num_train))
train_idx , valid_idx = idx[split:] , idx[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)




# train loader
train_loader = torch.utils.data.DataLoader(train_data , batch_size = 32 , sampler= train_sampler)

valid_loader = torch.utils.data.DataLoader(train_data , batch_size = 32 ,  sampler = valid_sampler)




In [6]:
train_data

Dataset ImageFolder
    Number of datapoints: 1500
    Root location: drive/My Drive/dataset/train/
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [7]:
t_img , t_label = next(iter(train_loader))
v_img , v_label = next(iter(valid_loader))

print("[INFO] Training data length.......: {}".format(len(train_data)))
t_label

[INFO] Training data length.......: 1500


tensor([0, 4, 4, 0, 4, 4, 0, 3, 4, 4, 4, 0, 1, 4, 1, 4, 4, 3, 1, 1, 4, 0, 2, 2,
        1, 0, 4, 0, 2, 1, 3, 0])

In [8]:
# load the pretrained model

vgg = models.vgg16(pretrained = True)

vgg

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
# if train_on_gpu:
#   print("CUDA")
#   vgg.cuda()

In [0]:
# freeze all parameters of network except classifier
for param in vgg.features.parameters():
	param.requires_grad = False

In [0]:
in_features = 25088

In [0]:
classifier = nn.Sequential(nn.Linear(in_features , 512),
							nn.ReLU(),
							nn.Linear(512 , 5))

vgg.classifier = classifier

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg.classifier.parameters() , lr = 0.01)

n_epochs = 4
total_train_loss, total_valid_loss = [], []
step = 0
accuracy = []
valid_loss_min = np.Inf

for e in range(n_epochs + 1):

  train_loss = 0
  valid_loss = 0

  vgg.train()
  for images, labels in train_loader:


    # if train_on_gpu:
    #   images , labels = images.cuda(), labels.cuda()

    optimizer.zero_grad()

    output = vgg(images)

    loss = criterion(output , labels)

    loss.backward()

    optimizer.step()

    train_loss += loss.item()*images.size(0)

    if step % 5 == 0:
      print("Step {}  \tTraining Loss: {:.4f}".format(step , train_loss))

    

    step += 1


  vgg.eval()
  for images, labels in valid_loader:

    optimizer.zero_grad()

    output = vgg(images)

    loss = criterion(output , labels)


    valid_loss += loss.item()*images.size(0)

   
  train_loss = train_loss / len(train_loader.sampler)
  valid_loss = valid_loss / len(valid_loader.sampler)


  total_train_loss.append(train_loss / len(train_loader.sampler))
  total_valid_loss.append(valid_loss / len(valid_loader.sampler))

  print("Epoch: {}, \tTraining Loss: {:.4f},    \tValidation Loss: {:.4f}".format(e + 1 , train_loss, valid_loss))


  # saving model
  if valid_loss <= valid_loss_min:
    print("Validation Loss Decreased ({:.6f} ----> {:.6f}). Saving Model........ ".format(valid_loss_min , valid_loss))

    torch.save(vgg.state_dict(), "drive/My Drive/dataset/weather_model.pt")
    valid_loss_min = valid_loss  


Step 0  	Training Loss: 12.2976
Step 5  	Training Loss: 71.3027
Step 10  	Training Loss: 122.7283
Step 15  	Training Loss: 170.3233
Step 20  	Training Loss: 228.7605
Step 25  	Training Loss: 271.8501
Step 30  	Training Loss: 318.1307
Step 35  	Training Loss: 364.2923
Epoch: 1, 	Training Loss: 0.3152,    	Validation Loss: 0.3205
Validation Loss Decreased (inf ----> 0.320497). Saving Model........ 
Step 40  	Training Loss: 30.1412
Step 45  	Training Loss: 70.1733
Step 50  	Training Loss: 112.4475
Step 55  	Training Loss: 150.1085
Step 60  	Training Loss: 195.7974
Step 65  	Training Loss: 234.1305
Step 70  	Training Loss: 272.3553
Step 75  	Training Loss: 305.4032
Epoch: 2, 	Training Loss: 0.2545,    	Validation Loss: 0.2682
Validation Loss Decreased (0.320497 ----> 0.268198). Saving Model........ 
Step 80  	Training Loss: 41.8869
Step 85  	Training Loss: 88.4175
Step 90  	Training Loss: 125.5027
Step 95  	Training Loss: 151.1063
Step 100  	Training Loss: 187.4018
Step 105  	Training Loss

In [19]:
# Load Modlel
vgg.load_state_dict(torch.load("drive/My Drive/dataset/weather_model.pt"))

<All keys matched successfully>

In [0]:
test_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
									                    transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])



test_data = datasets.ImageFolder("drive/My Drive/dataset/test" , transform= test_transform)

In [0]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size= 32)


In [66]:
import pandas as pd

dl = pd.read_csv("drive/My Drive/dataset/test.csv")

test_label = dl["labels"]

test_label = torch.tensor(test_label)
type(test_label)

torch.Tensor

In [67]:
test_loss = 0.0
cls_correct = 0


vgg.eval()

for data in test_loader:

  img, _ = data

  output = vgg(img)

  loss = criterion(output, test_label)

  test_loss += loss.item()*img.size(0)

  _, pred = torch.max(output, 1)

  pred = pred.numpy()

  for i in range(len(pred)):
    if test_label[i] == pred[i]:
      cls_correct += 1

test_loss = test_loss / len(test_loader)

acc = 100 * (cls_correct / len(test_label))

print("Accuracy: {}".format(acc))

Accuracy: 90.0


In [0]:
img, _ = data

In [0]:
cr = 0
for i in range(len(label)):
  if label[i] == prediction[i]:
    cr += 1